In [9]:
import os
import numpy as np
import pandas as pd
import sys
sys.path.append(os.path.abspath(".."))

from urlembed.util.seqmanager import *
from urlembed.util.plotter import *
from urlembed.util.clustering_metrics import *
from urlembed.model.url2vec import *

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE

from sklearn.cluster import KMeans
from hdbscan import HDBSCAN

from gensim.models import Word2Vec

from itertools import tee

In [10]:
"""from urlembed.util.sequence_handler import *
from urlembed.util.sequence_manager import *
from urlembed.model.url2vec import *

mappi = get_urlmap("/home/chris/workspace/jupyter-notebook/url2vec/dataset/new/cs.illinois.edu.ListConstraint.words1000.depth10/urlsMap.txt")
cmappi = get_content_map("/home/chris/workspace/jupyter-notebook/url2vec/dataset/new/cs.illinois.edu.ListConstraint.words1000.depth10/vertex.txt")
seq = get_seq("/home/chris/workspace/jupyter-notebook/url2vec/dataset/new/cs.illinois.edu.ListConstraint.words1000.depth10/sequenceIDs.txt")

rm = RealMembership(fpath="/home/chris/workspace/jupyter-notebook/url2vec/dataset/manual-membership/urlToMembership.txt")
ground_truth = [int(rm.get_membership(mappi[k])) for k in mappi]
u2v = Url2Vec(mappi)
labels = u2v.train(sequences_list=seq, codecontent_map=cmappi)
u2v.test(ground_truth)"""

'from urlembed.util.sequence_handler import *\nfrom urlembed.util.sequence_manager import *\nfrom urlembed.model.url2vec import *\n\nmappi = get_urlmap("/home/chris/workspace/jupyter-notebook/url2vec/dataset/new/cs.illinois.edu.ListConstraint.words1000.depth10/urlsMap.txt")\ncmappi = get_content_map("/home/chris/workspace/jupyter-notebook/url2vec/dataset/new/cs.illinois.edu.ListConstraint.words1000.depth10/vertex.txt")\nseq = get_seq("/home/chris/workspace/jupyter-notebook/url2vec/dataset/new/cs.illinois.edu.ListConstraint.words1000.depth10/sequenceIDs.txt")\n\nrm = RealMembership(fpath="/home/chris/workspace/jupyter-notebook/url2vec/dataset/manual-membership/urlToMembership.txt")\nground_truth = [int(rm.get_membership(mappi[k])) for k in mappi]\nu2v = Url2Vec(mappi)\nlabels = u2v.train(sequences_list=seq, codecontent_map=cmappi)\nu2v.test(ground_truth)'

In [11]:
nocostraint_path = os.getcwd() + "/../dataset/new/cs.illinois.edu.ListConstraint.words1000.depth10/"
rmfile  = "/home/chris/workspace/jupyter-notebook/url2vec/dataset/manual-membership/urlToMembership.txt"

vertex_path      = nocostraint_path + "vertex.txt"
codecontent_map  = get_content_map(vertex_path)

map_path         = nocostraint_path + "urlsMap.txt"
codeurl_map      = get_urlmap(map_path)

sequences        = nocostraint_path + "sequenceIDs.txt"
seq              = get_sequences(sequences)

gt = GroundTruth(rmfile)
ground_truth = [int(gt.get_groundtruth(codeurl_map[code])) for code in codeurl_map]

u2v = Url2Vec(codeurl_map)

In [12]:
labels = u2v.train(sequences_list=seq, codecontent_map=codecontent_map)
print(len(labels))

1022


In [13]:
df = pd.DataFrame(u2v.test(ground_truth), index=set(ground_truth), columns=set(labels))
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,-1
0,0,0,0,0,0,0,0,13,0,0,0,0,0,0,0,5
1,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,2
2,0,0,0,0,0,0,0,0,0,49,0,0,0,19,16,85
3,0,0,0,0,28,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6
8,0,0,0,0,0,0,0,0,0,0,128,36,0,0,0,127
10,0,0,0,0,0,0,0,0,11,0,0,0,200,0,0,105
12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10
13,0,9,27,11,0,0,0,0,0,0,0,0,0,0,0,10


In [19]:
nocostraint_path = os.getcwd() + "/../dataset/new/cs.illinois.edu.ListConstraint.words1000.depth10/"
rmfile  = "/home/chris/workspace/jupyter-notebook/url2vec/dataset/manual-membership/urlToMembership.txt"
vertex_path      = nocostraint_path + "vertex.txt"
map_path         = nocostraint_path + "urlsMap.txt"
sequences        = nocostraint_path + "sequenceIDs.txt"

urlmap = get_urlmap(map_path)

u2v  = Url2Vec(urlmap)
u2v2 = Url2Vec(urlmap)

In [20]:
seq = get_sequences(sequences)
codecontent_map = get_content_map(vertex_path)

k_memb = u2v.train(algorithm=KMeans(n_clusters=15), sequences_list=seq, codecontent_map=codecontent_map)

In [22]:
seq2 = get_sequences(sequences)
hd_memb = u2v2.train(sequences_list=seq2, codecontent_map=codecontent_map)

In [25]:
gt = GroundTruth(rmfile)
ground_truth = [int(gt.get_groundtruth(codeurl_map[code])) for code in codeurl_map]

confusion_table = u2v.test(ground_truth)
pd.DataFrame(confusion_table, index=set(ground_truth), columns=set(u2v.labels_))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,0,0,0,0,18,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0
2,0,0,0,0,98,0,70,0,0,0,1,0,0,0,0
3,0,0,28,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0
8,178,0,0,0,0,0,1,0,0,0,25,0,44,43,0
10,0,0,0,28,0,0,2,0,0,255,30,0,0,1,0
12,1,0,0,0,0,0,0,0,0,1,7,0,0,1,0
13,0,0,0,0,0,0,0,11,0,0,0,0,0,0,46


In [26]:
confusion_table_hdbscan = u2v2.test(ground_truth)
pd.DataFrame(confusion_table_hdbscan, index=set(ground_truth), columns=set(u2v2.labels_))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,-1
0,0,0,0,0,0,0,0,13,0,0,0,0,0,0,0,5
1,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,2
2,0,0,0,0,0,0,0,0,0,49,0,0,0,19,16,85
3,0,0,0,0,28,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6
8,0,0,0,0,0,0,0,0,0,0,128,36,0,0,0,127
10,0,0,0,0,0,0,0,0,11,0,0,0,200,0,0,105
12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10
13,0,9,27,11,0,0,0,0,0,0,0,0,0,0,0,10


In [30]:
# k_memb  
labels_pred_k = map(int, k_memb)
labels_pred_h = hd_memb

In [34]:
print "Homogeneity:\t\t", metrics.homogeneity_score(ground_truth, labels_pred_k)
print "Completeness:\t\t", metrics.completeness_score(ground_truth, labels_pred_k)
print "V Measure Score:\t", metrics.v_measure_score(ground_truth, labels_pred_k)
print "adjusted rand score:\t", metrics.adjusted_rand_score(ground_truth, labels_pred_k)

Homogeneity:		0.84267348786
Completeness:		0.673923988675
V Measure Score:	0.748910487952
adjusted rand score:	0.629647812665


In [35]:
print "Homogeneity:\t\t", metrics.homogeneity_score(ground_truth, labels_pred_h) # best 1
print "Completeness:\t\t", metrics.completeness_score(ground_truth, labels_pred_h)  # best 1
print "V Measure Score:\t", metrics.v_measure_score(ground_truth, labels_pred_h) # best 1
print "adjusted rand score:\t", metrics.adjusted_rand_score(ground_truth, labels_pred_h) # best 1

Homogeneity:		0.61951973608
Completeness:		0.579917642121
V Measure Score:	0.599064913475
adjusted rand score:	0.294670101365
